In [1]:
import mysql.connector as connector
from mysql.connector import Error

from utils import get_default_connection_config, get_default_database, print_environment_info, print_rows

try:
    connection = connector.connect(**get_default_connection_config(database=get_default_database()))
    print_environment_info(connection)
    cursor = connection.cursor()
except Error as e:
    print(f'Error: {e.errno}: {e.msg}')

Connector version: 8.0.32
Server version: 8.0.32


In [2]:
cursor.execute("""CREATE OR REPLACE VIEW OrdersView AS
SELECT OrderID, Quantity, TotalCost as Cost FROM Orders
WHERE Quantity > 2
ORDER BY Cost DESC ;
""")

print_rows(
    title='Selecting all rows from OrdersView',
    cursor=cursor,
    query="SELECT * FROM OrdersView"
)

Selecting all rows from OrdersView
----------------------------------
OrderID - Quantity - Cost
15 - 4 - 394.08
2 - 4 - 324.40
7 - 3 - 295.56
3 - 4 - 246.40
5 - 3 - 211.38
13 - 4 - 51.88
1 - 3 - 34.86
8 - 3 - 34.86
11 - 3 - 34.86


In [3]:
# Task 2
query = """
SELECT c.CustomerID, c.FullName,
       o.OrderID, o.TotalCost as Cost,
       m.MenuName,
       mi.CourseName, mi.StarterName
FROM Customers c INNER JOIN Orders o INNER JOIN Menus m INNER JOIN MenuItems mi INNER JOIN MenusAndMenuItems mmi
ON
    o.CustomerID = c.CustomerID AND
    mmi.MenuID = m.MenuID AND
    mi.MenuItemID = mmi.MenuItemID AND
    mmi.MenusAndMenuItemsID = o.MenusAndMenuItemsID
WHERE
    o.TotalCost > 150
ORDER BY Cost;
"""

print_rows(
    title='Selecting rows from all joins',
    cursor=cursor,
    query=query
)

Selecting rows from all joins
-----------------------------
CustomerID - FullName - OrderID - Cost - MenuName - CourseName - StarterName
2 - Full Name 1 - 16 - 162.20 - Menu 1 - Course 0 - Starter 2 for Course 0
4 - Full Name 3 - 5 - 211.38 - Menu 2 - Course 0 - Starter 3 for Course 0
4 - Full Name 3 - 3 - 246.40 - Menu 3 - Course 0 - Starter 0 for Course 0
1 - Full Name 0 - 7 - 295.56 - Menu 1 - Course 3 - Starter 0 for Course 3
2 - Full Name 1 - 2 - 324.40 - Menu 1 - Course 0 - Starter 2 for Course 0
3 - Full Name 2 - 15 - 394.08 - Menu 1 - Course 3 - Starter 0 for Course 3


In [4]:

# Task 3
# The order by clause is to make output more readable
query = """
SELECT MenuName from Menus WHERE MenuID = ANY(
    SELECT m.MenuID
    FROM Orders o INNER JOIN MenusAndMenuItems mmi INNER JOIN Menus m ON
    mmi.MenuID = m.MenuID AND
    mmi.MenusAndMenuItemsID = o.MenusAndMenuItemsID
    WHERE Quantity > 2
) ORDER BY MenuName;
"""

print_rows(
    title='Menu names for which more than 2 orders have been placed',
    cursor=cursor,
    query=query
)

Menu names for which more than 2 orders have been placed
--------------------------------------------------------
MenuName
Menu 1
Menu 2
Menu 3


In [5]:
assert cursor.close()
assert connection.close() is None